In [1]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import cv2
import os
import math
import torch
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np
import matplotlib.colorbar as colorbar

import requests
from io import BytesIO
from PIL import Image
import numpy as np
from maskrcnn_benchmark.utils import cv2_util
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

from moviepy.editor import VideoFileClip

# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20*1.5, 12*1.5

from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.structures.keypoint import keypoints_to_heat_map
from maskrcnn_benchmark.modeling.roi_heads.keypoint_head.loss import project_keypoints_to_heatmap
from demo.predictor import COCODemo
from torchvision import transforms as T

from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.structures.image_list import to_image_list
from maskrcnn_benchmark.modeling.roi_heads.mask_head.inference import Masker
from maskrcnn_benchmark import layers as L
from maskrcnn_benchmark.utils import cv2_util

from maskrcnn_benchmark.structures.keypoint import PersonKeypoints, BeeKeypoints, FlyKeypoints


In [5]:
!find .  | xargs make_data

xargs: make_data: No such file or directory
find: ‘standard output’: Broken pipe
find: write error


In [2]:
def load(img_path):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    pil_image = Image.open(img_path).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def load_video(video_path,frame_id=[0]):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    
    video_clip = VideoFileClip(video_path)
    if frame_id is None:
        nframes = int(video_clip.fps * video_clip.duration)
        frame_id = range(nframes)
    images = []
    for i in frame_id:
        image = video_clip.get_frame(i/video_clip.fps)
        # convert to BGR format
        image = np.array(image)[:, :, [2, 1, 0]]
        images += [image]
    video_clip.close()
    
    return images

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    
def sigmoid(x):
  return 1 / (1 + math.exp(-x))
# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)

# num keypoints per animal, heatmap
def calculate_peaks(numparts, heatmap_avg):
    score = np.ones((numparts,)) * 0.000001
    all_peaks = []
    peak_counter = 0
    if len(score) < numparts:
        score = score[:numparts]
        print("score", score)
        ##logger.ERROR(‘Not enough scores provided for number of parts’)
        # return
    # threshold_detection = params[‘thre1’]
    # tic_localmax=time.time()
    for part in range(numparts):
        map_ori = heatmap_avg[part, :, :]
        map = map_ori
        map_left = np.zeros(map.shape)
        map_left[1:, :] = map[:-1, :]
        map_right = np.zeros(map.shape)
        map_right[:-1, :] = map[1:, :]
        map_up = np.zeros(map.shape)
        map_up[:, 1:] = map[:, :-1]
        map_down = np.zeros(map.shape)
        map_down[:, :-1] = map[:, 1:]
        #peaks_binary = np.logical_and(np.logical_and(np.logical_and(map >= map_left, map >= map_right),
                                                     #np.logical_and(map >= map_up, map >= map_down)), map > score[part])
        peaks_binary = (sigmoid_v(map_ori) > .2)
        print("pb shap", np.shape(peaks_binary))
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0]))  # note reverse
        peaks_with_score_and_id = [x + (map_ori[x[1], x[0]], i + peak_counter,) for i, x in
                                   enumerate(peaks)]  # if x[0]>0 and x[1]>0 ]
        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)
    return all_peaks

def load_images_from_folder(folder):
    images = []
    file_names = []
    for filename in os.listdir(folder):
        if ".jpg" in filename:
            img = load(os.path.join(folder,filename))
            if img is not None:
                images.append(img)
                file_names.append(filename)
    return images, file_names


In [3]:
params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'image.cmap': 'jet',
    'axes.grid': False,
    'savefig.dpi': 150,  # to adjust notebook inline plot size
    'axes.labelsize': 10, # fontsize for x and y labels (was 10)
    'axes.titlesize': 12,
    'font.size': 12, # was 10
    'legend.fontsize': 10, # was 10
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    #'text.usetex': True,
    'figure.figsize': [3.39, 2.10],
    'font.family': 'serif',
}
matplotlib.rcParams.update(params)

In [4]:
def overlay_keypoints(image, predictions):
        #keypoints = predictions.get_field("keypoints")
        kps = predictions#keypoints.keypoints
        scores = kps.new_ones((kps.size(0), kps.size(1)))
        kps = torch.cat((kps[:, :, 0:2], scores[:, :, None]), dim=2).numpy()
        for region in kps:
            kfun = BeeKeypoints
            image = vis_keypoints_others(
                    image,
                    region.transpose((1, 0)),
                    kp_thresh=0,
                    kfun=kfun)
                
        return image


In [5]:
def vis_keypoints_others(img, kps, kp_thresh=2, alpha=0.7, kfun=PersonKeypoints):
    """Visualizes keypoints (adapted from vis_one_image).
    kps has shape (4, #keypoints) where 4 rows are (x, y, logit, prob).
    """
    img = img.copy()
    dataset_keypoints = kfun.NAMES
    kp_lines = kfun.CONNECTIONS

    # Convert from plt 0-1 RGBA colors to 0-255 BGR colors for opencv.
    cmap = plt.get_cmap('rainbow')
    colors = [cmap(i) for i in np.linspace(0, 1, len(kp_lines) + 2)]
    colors = [(c[2] * 255, c[1] * 255, c[0] * 255) for c in colors]

    # Perform the drawing on a copy of the image, to allow for blending.
    kp_mask = np.copy(img)
    
    # Draw the keypoints.
    for l in [0]:#range(len(kp_lines)):
        i1 = kp_lines[l][0]
        i2 = kp_lines[l][1]
        p1 = kps[0, i1], kps[1, i1]
        p2 = kps[0, i2], kps[1, i2]
        #if kps[2, i1] > kp_thresh and kps[2, i2] > kp_thresh:
         #   cv2.line(
          #      kp_mask, p1, p2,
           #     color=colors[l], thickness=2, lineType=cv2.LINE_AA)
        if kps[2, i1] > kp_thresh:
            cv2.circle(
                kp_mask, p1,
                radius=16, color=colors[l], thickness=-1, lineType=cv2.LINE_AA)
        #if kps[2, i2] > kp_thresh:
         #   cv2.circle(
          #      kp_mask, p2,
           #     radius=16, color=colors[l], thickness=-1, lineType=cv2.LINE_AA)

    # Blend the keypoints.
    return cv2.addWeighted(img, 1.0 - alpha, kp_mask, alpha, 0)

In [6]:
#test_file = 'test_bee_2.png'
#image = load(test_file)

In [7]:
folder = "datasets/bee/train/"
test_images, test_image_files = load_images_from_folder(folder)

In [8]:
#need to get targets from images
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.config import cfg
config_file = "/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml"
cfg.merge_from_file(config_file)
cfg.merge_from_list(['DATALOADER.NUM_WORKERS', 2, \
                     'DATATYPE', 'bee', \
        'OUTPUT_DIR', 'training_dir/250/l_standard/fcos_kps_ms_training_R_50_FPN_1x_bee', \
        'SOLVER.KPS_GRAD_MULT', 10.0, \
        'SOLVER.MAX_GRAD_NORM', 5.0, \
        'SOLVER.POWER', 1.0, \
        'INPUT.CROP_SIZE', 800, \
        'INPUT.MIN_SIZE_RANGE_TRAIN', '(480, 1600)', \
        'INPUT.MAX_SIZE_TRAIN', 2666, \
        'MODEL.WEIGHT', 'training_dir/250/standard/fcos_kps_ms_training_R_50_FPN_1x_bee/model_final.pth', \
        'MODEL.RPN.BATCH_SIZE_PER_IMAGE', 1,\
        'MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE', 1,\
        'SOLVER.IMS_PER_BATCH', 3, \
        'SOLVER.MAX_ITER', 180000, \
        'MODEL.HEATMAPS_LOSS_WEIGHT', 4.0, \
        'DATASETS.TRAIN', "('bee_train_cocostyle', 'bee_train_cocostyle')"
        ])

data_loader, dataset = make_data_loader(
        cfg,
        start_iter=0,
    )

hoo
DATA Path;  ('bee_train_cocostyle', 'bee_train_cocostyle')
data? : {'factory': 'COCODataset', 'args': {'root': 'datasets/bee/train', 'ann_file': 'datasets/bee/annotations/train_bee_annotations2018_nondup.json', 'kfun': 'BeeKeypoints'}}
fact:  <class 'maskrcnn_benchmark.data.datasets.coco.COCODataset'>
D:  <module 'maskrcnn_benchmark.data.datasets' from '/share/ctn/users/bsb2144/directpose/maskrcnn_benchmark/data/datasets/__init__.py'>
args:  {'root': 'datasets/bee/train', 'ann_file': 'datasets/bee/annotations/train_bee_annotations2018_nondup.json', 'kfun': 'BeeKeypoints', 'remove_images_without_annotations': True, 'transforms': Compose(
)}
name?
datasets/bee/annotations/train_bee_annotations2018_nondup.json
datasets/bee/train
BeeKeypoints
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
ids: : [1706210, 1706220, 1706230, 1706211200, 1706211912, 1706212400, 1706212912, 1706213912, 1706214912, 1706216112, 1706217312, 1706218312, 1706219512, 170622120

In [9]:
coco_demo = COCODemo(
                cfg,
                min_image_size=800,
                confidence_threshold=0.2,
            )


In [10]:
#print(dataset.id_to_img_map)
for i, d in enumerate(data_loader): # or i, image in enumerate(dataset)
    print(d)
    print("id: ", dataset.get_img_info(i)["file_name"] )
    print("next")

(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e0143c8>, (BoxList(num_boxes=13, image_width=1333, image_height=750, mode=xyxy),), (0,))
id:  000001706210.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e014240>, (BoxList(num_boxes=11, image_width=1333, image_height=750, mode=xyxy),), (1,))
id:  000001706220.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e01cc88>, (BoxList(num_boxes=24, image_width=1333, image_height=750, mode=xyxy),), (2,))
id:  000001706230.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e00a400>, (BoxList(num_boxes=10, image_width=1333, image_height=750, mode=xyxy),), (3,))
id:  001706211200.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e025a58>, (BoxList(num_boxes=7, image_width=1333, image_height=750, mode=xyxy),), (4,))
id:  001706211912.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e

(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e03fc50>, (BoxList(num_boxes=9, image_width=1333, image_height=750, mode=xyxy),), (44,))
id:  017062120464.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e0149e8>, (BoxList(num_boxes=12, image_width=1333, image_height=750, mode=xyxy),), (45,))
id:  017062121664.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e02f2e8>, (BoxList(num_boxes=8, image_width=1333, image_height=750, mode=xyxy),), (46,))
id:  017062123264.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b57442e4d68>, (BoxList(num_boxes=11, image_width=1333, image_height=750, mode=xyxy),), (47,))
id:  017062124664.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e0149b0>, (BoxList(num_boxes=8, image_width=1333, image_height=750, mode=xyxy),), (48,))
id:  017062125864.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b5

(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e0364e0>, (BoxList(num_boxes=17, image_width=1333, image_height=750, mode=xyxy),), (88,))
id:  017062222400.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e01c320>, (BoxList(num_boxes=12, image_width=1333, image_height=750, mode=xyxy),), (89,))
id:  017062223600.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b57442e4ba8>, (BoxList(num_boxes=16, image_width=1333, image_height=750, mode=xyxy),), (90,))
id:  017062224800.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b57442e4b38>, (BoxList(num_boxes=18, image_width=1333, image_height=750, mode=xyxy),), (91,))
id:  017062226000.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e02f320>, (BoxList(num_boxes=11, image_width=1333, image_height=750, mode=xyxy),), (92,))
id:  017062227200.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x

(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e01c1d0>, (BoxList(num_boxes=23, image_width=1333, image_height=750, mode=xyxy),), (132,))
id:  017062325200.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e0362e8>, (BoxList(num_boxes=22, image_width=1333, image_height=750, mode=xyxy),), (133,))
id:  017062326400.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b57442e4ef0>, (BoxList(num_boxes=26, image_width=1333, image_height=750, mode=xyxy),), (134,))
id:  017062327600.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e0146a0>, (BoxList(num_boxes=13, image_width=1333, image_height=750, mode=xyxy),), (0,))
id:  000001706210.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e025a58>, (BoxList(num_boxes=11, image_width=1333, image_height=750, mode=xyxy),), (1,))
id:  000001706220.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0

(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e01c320>, (BoxList(num_boxes=6, image_width=1333, image_height=750, mode=xyxy),), (41,))
id:  017062114312.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e02fbe0>, (BoxList(num_boxes=12, image_width=1333, image_height=750, mode=xyxy),), (42,))
id:  017062115512.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b57442e4d68>, (BoxList(num_boxes=8, image_width=1333, image_height=750, mode=xyxy),), (43,))
id:  017062117112.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e02f6d8>, (BoxList(num_boxes=9, image_width=1333, image_height=750, mode=xyxy),), (44,))
id:  017062120464.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e01c978>, (BoxList(num_boxes=12, image_width=1333, image_height=750, mode=xyxy),), (45,))
id:  017062121664.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b5

(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e01c278>, (BoxList(num_boxes=15, image_width=1333, image_height=750, mode=xyxy),), (86,))
id:  017062220000.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e03f278>, (BoxList(num_boxes=8, image_width=1333, image_height=750, mode=xyxy),), (87,))
id:  017062221200.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e0146a0>, (BoxList(num_boxes=17, image_width=1333, image_height=750, mode=xyxy),), (88,))
id:  017062222400.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e01c898>, (BoxList(num_boxes=12, image_width=1333, image_height=750, mode=xyxy),), (89,))
id:  017062223600.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e03f1d0>, (BoxList(num_boxes=16, image_width=1333, image_height=750, mode=xyxy),), (90,))
id:  017062224800.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2

(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b57442e4eb8>, (BoxList(num_boxes=16, image_width=1333, image_height=750, mode=xyxy),), (130,))
id:  017062322800.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e014cf8>, (BoxList(num_boxes=19, image_width=1333, image_height=750, mode=xyxy),), (131,))
id:  017062324000.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e02fc18>, (BoxList(num_boxes=23, image_width=1333, image_height=750, mode=xyxy),), (132,))
id:  017062325200.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e025630>, (BoxList(num_boxes=22, image_width=1333, image_height=750, mode=xyxy),), (133,))
id:  017062326400.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object at 0x2b574e014198>, (BoxList(num_boxes=26, image_width=1333, image_height=750, mode=xyxy),), (134,))
id:  017062327600.jpg
next
(<maskrcnn_benchmark.structures.image_list.ImageList object 

IndexError: list index out of range

In [ ]:
print(len(data_loader))
#print(dataset.id_to_img_map)
#for i, d in enumerate(data_loader): # or i, image in enumerate(dataset)
 #   print(d)
  #  print("id: ", dataset.get_img_info(0)["file_name"] )
    #print("next")
i =0
c_losses = []


for iteration, (images, targets, _) in enumerate(data_loader, i):
        i = i + 1
        #print(images)
        #print([targets[0]])
        t = targets[0].to("cuda")
        tt = [t]
        images = images.to("cuda")
        ##tt = tt.cuda()
        ##predictions, tps, hm_res, stride, hms, targets, c_loss = coco_demo.run_on_opencv_image(images, tt)
        c_loss = coco_demo.run_on_opencv_image(images, tt)
        c_losses.append(c_loss)
        print("c loss: ", c_loss)
        print("next")

#print(c_losses)
print(len(c_losses))
print(c_losses[0])
print(c_losses[1])
print(c_losses[2])
print(c_losses[3])
print("brek")
print(c_losses[135])
print(c_losses[136])
print(c_losses[137])
print(c_losses[138])

import numpy as np
cln = c_losses[0:135]
for i, item in enumerate(cln):
    cln[i] = item.item()
print(len(cln))
print(cln[0])
arr = np.array(cln)
max_ids = (arr.argsort()[-10:][::-1])
print(max_ids)
print(arr)

max_files = []
for ind in max_ids:
    file = dataset.get_img_info(ind)["file_name"]
    max_files.append(file)
    
print(max_files) 

In [ ]:
import matplotlib.gridspec as gridspec

pad = 5
k=10
#print("test ig: ", test_images)
#print("test files: ", test_image_files)
rows = ["Heatmap Loss", "Original DP Loss", "Combo Loss", "Combo OG"]#"DP Predictions", "Combined Heatmap", "Combined Pred"]
cols = ["Heatmap", "Predictions"]
data_size = "250"
version = "0"
loss = "combined_from_standard_nw"



In [1]:
fig, axes = plt.subplots(1,3,figsize=(20,12),constrained_layout=True,gridspec_kw = {'wspace':.15, 'hspace':.15})
fig.subplots_adjust(wspace=.15, hspace=.000001)

title = 'Data Set: ' + data_size
fig.suptitle(title, y=.92)

config_file = "/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml"

cfg.merge_from_file("/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml")
cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                     'DATATYPE', 'bee', \
                     'OUTPUT_DIR', 'training_dir/' + data_size + '/' + loss +'/fcos_kps_ms_training_R_50_FPN_1x_bee1', \
                     'MODEL.WEIGHT', '/home/bsb2144/directpose/tools/training_dir/' + data_size + '/'+ loss +'/fcos_kps_ms_training_R_50_FPN_1x_bee/model_final.pth',\
                     'DATASETS.TEST', "('bee_val_cocostyle',)",\
                     'DATASETS.TRAIN', "('bee_train_cocostyle', )",\
                    ])

coco_demo = COCODemo(
            cfg,
            min_image_size=800,
            confidence_threshold=0.2,
)

c_losses = []

for idx, image in enumerate(test_images):
    predictions, tps, hm_res, stride, hms, targets, c_loss = coco_demo.run_on_opencv_image(image)
    c_losses.append(c_loss)
    




NameError: name 'plt' is not defined

1 of 8: 100/0/standard 

In [ ]:
sigmoid(0.37848875)